In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
#imports
import os
import h5py
import scipy
import math
from sklearn import metrics
import numpy as np
import pandas as pd 
from IPython.display import Audio
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib.pyplot import imread
import librosa
import librosa.display
!pip install natsort
import natsort
from scipy import signal
from scipy.io import wavfile
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops


%matplotlib inline
np.random.seed(1)

In [13]:
data_path = "../input/speech-emotion-recognition-en/Crema"

In [14]:
def load_data() :
    labels = []
    mel_spectograms=[]
    features=[]

    for file in natsort.natsorted(os.listdir(data_path)):
        fileName = file.split("_")
        label=fileName[2]
        path = os.path.join(data_path , file)
        sound, sr = librosa.load(path, duration=3.5, sr=None)
        sound_noise = noise(sound, random = True)
        #sound_pitch = pitch(sound, sr, pitch_factor=0.5, random=True)
        if librosa.get_duration(y = sound) < 3.5:
          sound = np.pad(sound,(math.ceil((77175-sound.shape[0])/2),math.floor((77175-sound.shape[0])/2)), 'constant')
          sound_noise = np.pad(sound_noise,(math.ceil((77175-sound_noise.shape[0])/2),math.floor((77175-sound_noise.shape[0])/2)), 'constant')
          #sound_pitch = np.pad(sound_pitch,(math.ceil((77175-sound_pitch.shape[0])/2),math.floor((77175-sound_pitch.shape[0])/2)), 'constant')
       
        mel_spectograms.append(mel_spectogram(sound,sr))
        mel_spectograms.append(mel_spectogram(sound_noise,sr))
        #mel_spectograms.append(mel_spectogram(sound_pitch,sr))
        
        features.append(np.append(zero_crossingRate(sound), energy(sound)))
        features.append(np.append(zero_crossingRate(sound_noise), energy(sound_noise)))
        #features.append(np.append(zero_crossingRate(sound_pitch), energy(sound_pitch)))
        
        labels.extend([label]*2)
        
    return features, mel_spectograms, labels

In [15]:
def load_files(i,j):
  files = sorted(os.listdir(data_path))
  paths=[]
  for i in range(i,j):
    path = os.path.join(data_path , files[i])
    paths.append(path)
  return paths

In [16]:
def noise(data, random=False, rate=0.035, threshold=0.075):
    """Add some noise to sound sample. Use random if you want to add random noise with some threshold.
    Or use rate Random=False and rate for always adding fixed noise."""
    if random:
        rate = np.random.random() * threshold
    noise_amp = rate*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def pitch(data, sampling_rate, pitch_factor=0.7, random=False):
    """"Add some pitch to sound sample. Use random if you want to add random pitch with some threshold.
    Or use pitch_factor Random=False and rate for always adding fixed pitch."""
    if random:
        pitch_factor=np.random.random() * pitch_factor
    return librosa.effects.pitch_shift(data, sr = sampling_rate, n_steps = pitch_factor)

In [17]:
def mel_spectogram(sound,sr):
    mel_spect = librosa.feature.melspectrogram(y=sound, sr=sr)
    return np.expand_dims(mel_spect,2)

In [18]:
def zero_crossingRate(sound):
  return librosa.feature.zero_crossing_rate(y=sound)

In [19]:
def energy(sound):
   return librosa.feature.rms(y=sound)

In [20]:
features, mel_spectograms, labels = load_data()

In [21]:
def get_splitted_data(data, labels):
    # we startify the data corresponding to label as the should be dsitributed on all the splitted data
    
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.3, random_state=0,stratify = labels)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.05,random_state=0, stratify = y_train)

    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test), np.array(X_val), np.array(y_val)

In [22]:
x_train_2d, y_train_2d, X_test_2d, y_test_2d, X_val_2d, y_val_2d =  get_splitted_data(mel_spectograms, labels.copy())

In [23]:
# labels ['ANG' 'DIS' 'FEA' 'HAP' 'NEU' 'SAD']
LABELS = {'ANG' : np.array([1, 0, 0, 0, 0, 0]),
          'DIS' : np.array([0, 1, 0, 0, 0, 0]),
          'FEA' : np.array([0, 0, 1, 0, 0, 0]),
          'HAP' : np.array([0, 0, 0, 1, 0, 0]),
          'NEU' : np.array([0, 0, 0, 0, 1, 0]),
          'SAD' : np.array([0, 0, 0, 0, 0, 1])
        }

In [24]:
y_train_2d = np.array([LABELS[letter] for letter in y_train_2d])
y_test_2d = np.array([LABELS[letter] for letter in y_test_2d])
y_val_2d = np.array([LABELS[letter] for letter in y_val_2d])
print(y_train_2d.shape)
print(y_test_2d.shape)
print(y_val_2d.shape)

(9897, 6)
(4466, 6)
(521, 6)


In [25]:
# 2D
x_train_2d = (x_train_2d - np.min(x_train_2d)) / (np.max(x_train_2d) - np.min(x_train_2d))
X_test_2d = (X_test_2d - np.min(X_test_2d)) / (np.max(X_test_2d) - np.min(X_test_2d))
X_val_2d = (X_val_2d - np.min(X_val_2d)) / (np.max(X_val_2d) - np.min(X_val_2d))

In [26]:
def create_conv_model_2d(input_shape):
  input_img = tf.keras.Input(shape=input_shape)
  Z0 = tfl.Conv2D(256, 5, activation='relu', padding="same", strides=1)(input_img)
  Z1 = tfl.Conv2D(128, 5, activation='relu', padding="same", strides=1)(Z0)
  del(Z0)
  P1 = tfl.MaxPool2D(pool_size=(5, 5), strides=(2, 2), padding='same')(Z1)
  del(Z1)
  Z2 = tfl.Conv2D(128, 5, activation='relu', padding="same", strides=1)(P1)
  del(P1)
  P2 = tfl.MaxPool2D(pool_size=(5, 5), strides=(2, 2), padding='same')(Z2)
  del(Z2)
  Z3 = tfl.Conv2D(128, 5, activation='relu', padding="same", strides=1)(P2)
  del(P2)
  P3 = tfl.MaxPool2D(pool_size=(5, 5), strides=(2, 2), padding='same')(Z3)
  del(Z3)
  F = tfl.Flatten()(P3)
  del(P3)
  FC1 = tfl.Dense(128, activation='relu')(F)
  del(F)
  FC2 = tfl.Dense(100, activation='relu')(FC1)
  FC3 = tfl.Dense(10, activation='relu')(FC2)
  del(FC2)
  outputs = tfl.Dense(6, activation='softmax')(FC1)
  del(FC1)
    
  # YOUR CODE ENDS HERE
  model = tf.keras.Model(inputs=input_img, outputs=outputs)
  return model

In [27]:
conv_model_2d = create_conv_model_2d((128, 151, 1))
opt3 = tf.keras.optimizers.Adam(learning_rate=0.0001)
conv_model_2d.compile(optimizer=opt3,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model_2d.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 151, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 151, 256)     6656      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 151, 128)     819328    
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 76, 128)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 76, 128)       409728    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 38, 128)      0         
 2D)                                                         

In [ ]:
train_dataset_2d = tf.data.Dataset.from_tensor_slices((x_train_2d, y_train_2d)).batch(64)
test_dataset_2d = tf.data.Dataset.from_tensor_slices((X_val_2d, y_val_2d)).batch(64)
checkpoint_filepath_2d = '/content/gdrive/MyDrive/pr/Assignment_3/checkpoint_2dModel1FINAL'
model_checkpoint_callback_2d = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath_2d,save_weights_only=True,monitor='val_accuracy',mode='max',save_best_only=True)
history = conv_model_2d.fit(train_dataset_2d, epochs=90, validation_data=test_dataset_2d, callbacks=[model_checkpoint_callback_2d])

Epoch 1/90
155/155 [==============================] - 132s 626ms/step - loss: 1.6233 - accuracy: 0.3092 - val_loss: 1.5612 - val_accuracy: 0.3436
Epoch 2/90
 29/155 [====>.........................] - ETA: 1:09 - loss: 1.5404 - accuracy: 0.3534

In [ ]:
test_dataset_2d = tf.data.Dataset.from_tensor_slices((X_test_2d, y_test_2d)).batch(64)
conv_model_2d.load_weights(checkpoint_filepath_2d)
conv_model_2d.evaluate(test_dataset_2d)